In [1]:
import numpy as np

from numpy.linalg import norm

# A word on global variables and closures

Consider the following (simplified, and with wrong formulas) design, inspired by TP1:

In [2]:
def simu_linreg():
    np.random.seed(24)  # seed to always get the same A and b
    A = np.random.randn(10, 10)
    b = A.dot(np.random.randn(10)) # values in ]- \infty, \infty[]
    return A, b
    
def simu_logreg():
    np.random.seed(24)  # seed to always get the same A and b
    A = np.random.randn(10, 10)
    b = np.sign(A.dot(np.random.randn(10)))  # values exactly -1 or 1
    return A, b

def loss_linreg(x):
    return norm(A.dot(x) - b)

def loss_logreg(x):
    return norm(np.exp(A.dot(x) * b))

It is a bad design, because loss_linreg and loss_logreg use two global variables, `A` and `b`, which are not meant to be the same (in the TP case, for linreg b has values in $[-\infty, + \infty]$.
                                                                                                                     whereas for logreg b has values in {-1, 1}.
                                                                                                                     
                                                                                                                     
`A` and `b` must be defined as global variables before we call loss_linreg():

In [3]:
x = np.arange(10)
loss_linreg(x)

NameError: name 'A' is not defined

So we fix that by instantiating A and b:

In [4]:
A, b = simu_linreg()
loss_linreg(x)

64.08451461074556

In [5]:
# we do it for logreg too:
A, b = simu_logreg()
loss_logreg(x)

319044211109848.7

So now everything works fine? No: if we call again `loss_linreg()`, A and b now refer to variables generated by `simu_logreg()` in the previous cell, so the value is not the old one:

In [7]:
loss_linreg(x)  # slightly different from what we had earlier

65.18082850239307

The hack that you saw in class is to define different variables for linreg and logreg so that the name won't clash:

In [9]:
def loss_linreg_v1(x):
    return norm(A_linreg.dot(x) - b_linreg)

A_linreg, b_linreg = simu_linreg()
A_logreg, b_logreg = simu_logreg()
loss_linreg_v1(x)  # the good one

64.08451461074556

But this is a bit ugly. The clean solution #1 is to use A and b as parameters:

In [10]:
def loss_linreg_v2(x, A, b):
    return norm(A.dot(x) - b)

But then you can't just call loss_linreg_v2(x), and passing A and b to ISTA and FISTA is a bit heavy:

In [11]:
loss_linreg_v2(x)

TypeError: loss_linreg_v2() missing 2 required positional arguments: 'A' and 'b'

You don't want to do this:

In [12]:
def loss_linreg_v2(x):
    A, b = simu_linreg()
    return norm(A.dot(x) - b)

because you would create a new dataset everytime you compute the loss

Solution #2 (cleanest one) is to use a closure: a function which defines a function inside, and returns this function:

In [13]:
def create_loss_linreg():
    A, b = simu_linreg()  # now A and b are defined in this scope, not as global variables.
    # /!\ A and b could also be arguments of create_loss_linreg().
    
    # I define a function inside the closure
    def loss_linreg(x):
        return norm(A.dot(x) - b)
    
    # I return this function:
    return loss_linreg


loss_linreg_v3 = create_loss_linreg()  # create_loss_linreg returns a function
print(loss_linreg_v3(x))

# now changing A and b as global variables doesn't affect the function:
A, b = simu_logreg()
print(loss_linreg_v3(x))  # the same value, it has not changed

64.08451461074556
64.08451461074556


As a conclusion:
- either you go with the original design, and you *make sure to redefine A and b* when you change between algorithms
- either you create a closure, and you recreate a new loss_function with it everytime you change algorithm or design